### All the transform functions

In [1]:
from torchvision import transforms
transform2 = transforms.Compose([#vgg16 
    transforms.Resize((299, 299)),
    transforms.ToTensor(), # ToTensor image ko convert karta hai pixelated values ye pytorch tensor me ie [0,1]
    transforms.Normalize(  #Normalize scale karta hai image ko [-1,1] me    
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

transform3 = transforms.Compose([          #resnet
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]
                         )
])

transform4 = transforms.Compose([          #mobilenetv2 
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

### Testing all the transform functions


In [ ]:
# Testing each transform on a sample image
import matplotlib.pyplot as plt
from torchvision import datasets
from PIL import Image

# Load a sample image from the dataset
sample_img_path = "C:\\Users\\hardi\\Downloads\\archive\\MepcoTropicLeaf-V1\\Database"
sample_img = datasets.ImageFolder(sample_img_path).imgs[0][0]  # Get first image path
img = Image.open(sample_img)

print(sample_img)

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(5, 5))
fig.suptitle('Comparing Different Transforms')

# Original image
axes[0,0].imshow(img)
axes[0,0].set_title('Original Image')
axes[0,0].axis('off')

# Transform 2
img_t2 = transform2(img)
img_t2 = img_t2.permute(1,2,0).numpy()  # Convert from CxHxW to HxWxC
img_t2 = (img_t2 * 0.5 + 0.5).clip(0, 1)  # Denormalize
axes[0,1].imshow(img_t2)
axes[0,1].set_title('Transform 2 (299x299)')
axes[0,1].axis('off')

# Transform 3
img_t3 = transform3(img)
img_t3 = img_t3.permute(1,2,0).numpy()
img_t3 = (img_t3 * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]).clip(0, 1)  # Denormalize
axes[1,0].imshow(img_t3)
axes[1,0].set_title('Transform 3 (299x299)')
axes[1,0].axis('off')

# Transform 4
img_t4 = transform4(img)
img_t4 = img_t4.permute(1,2,0).numpy()
img_t4 = (img_t4 * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]).clip(0, 1)  # Denormalize
axes[1,1].imshow(img_t4)
axes[1,1].set_title('Transform 4 (224x224)')
axes[1,1].axis('off')

plt.tight_layout()
plt.show()

### Importing the Dataset


In [ ]:
from torchvision import datasets
# from google.colab import drive
# drive.mount('/content/drive')
# dataset_vgg=datasets.ImageFolder("/content/drive/MyDrive/patta /archive/MepcoTropicLeaf-V1/Database", transform=transform2)
# dataset_mobilenet=datasets.ImageFolder("/content/drive/MyDrive/patta /archive/MepcoTropicLeaf-V1/Database", transform=transform4)
# dataset_resnet=datasets.ImageFolder("/content/drive/MyDrive/patta /archive/MepcoTropicLeaf-V1/Database", transform=transform3)
dataset_vgg=datasets.ImageFolder("C:\\Users\\hardi\\Downloads\\archive\\MepcoTropicLeaf-V1\\Database", transform=transform2)
dataset_mobilenet=datasets.ImageFolder("C:\\Users\\hardi\\Downloads\\archive\\MepcoTropicLeaf-V1\\Database", transform=transform4)
dataset_resnet=datasets.ImageFolder("C:\\Users\\hardi\\Downloads\\archive\\MepcoTropicLeaf-V1\\Database", transform=transform3)
print(dataset_vgg)
print(dataset_mobilenet)
print(dataset_resnet)

### Spliting the Dataset

In [ ]:
import numpy as np
indices=list(range(len(dataset_vgg))) # dataset ko numerate karne ke liye
split=int(np.floor(0.70*len(dataset_vgg)))
validation=int(np.floor(0.60*split))

# agar tmre pass 100 samples hai
# toh split=70(ie 70% of the dataset)-> ee use hoga training and validation ke liye; remaining (30)30% used hoga as test set
# tb validation=42 (ie 60% of the dataset)-> ee use hoga for training ke liye; remaining (28)40% used hoga as Validation set 
# toh phir training= 42%; validation= 28%; test=30%

print(f"length of train size : {validation}")  
print(f"length of validation size : {split-validation}")  
print(f"length of test size : {len(dataset_vgg)-split}")  

np.random.shuffle(indices) # dataset me randomness laane ke liye

# ab actual splitting 
train_indices, validation_indices, test_indices = (
    indices[:validation], # [:5]->0,1,2,3,4
    indices[validation:split],# [1:3]->1,2
    indices[split:],# [2:]-> 2,3,4,5,........
)

from torch.utils.data.sampler import SubsetRandomSampler #randomly shuffles the dataset
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

# print(list(train_indices))
# print(list(validation_indices))
# print(list(test_indices))

# print(list(train_sampler))
# print(list(validation_sampler))
# print(list(test_sampler))


In [ ]:
targets_size = len(dataset_vgg.class_to_idx)# finding the total unique classes and storing it
print(targets_size)
print(list(dataset_vgg.class_to_idx.keys()))
num_classes_list = list(dataset_vgg.class_to_idx.values())# now numerating them
print(num_classes_list)

### Loading the models

In [ ]:
from torchvision import models
model = models.vgg16(pretrained=True)#if pretrained=false, tb saare layers ko hmlog ko individually train karna hoga
model2=models.mobilenet_v2(pretrained=True)
model3=models.resnet18(pretrained=True)

# model 
# model2
# model3 

for params in model3.parameters(): 
   params.requires_grad = True
# resnet18 ke saare parameters ko trainable bana rhe hai; resnet ka feature extraction acha kaam karra hai isliye use krre plus accuracy bhi increase ho rha  
# mobilenet aur vgg complex models hai (more param and layers), agar isko(function) use kare toh model overfit ho sakta 

### Extracting feature size 

In [ ]:
n_features = model.classifier[0].in_features #vgg16; number of input features in the first fully connected layer
print(1, n_features)
n_features = model2.classifier[1].in_features #mobilenet; number of input features in the first fully connected layer
print(2, n_features)
n_features = model3.conv1.in_channels #resnet18; number of input features in the first convolutional layer
print(3, n_features)

# ee kyu krre?-> taaki pata chale ki input and output features pata chale
# Last layer change karke model ko apne data pe train karre hai.  
# Pehle wali layers ko rakhne ka fayda->  
# Ye layers already trained hota h aur feature extraction achha kar leta h.  
# Sirf last layer train karne se kam computation time lagega.  
# Kam data hone par bhi model achha perform karega, kyunki pehle se trained layers use ho rahi hain.  
# Isko *Transfer Learning kehte hain 

### Over-riding the calculations to GPU(if available)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) #checking if GPU is available
model.to(device)
model2.to(device)
model3.to(device)#shifting the model to 'device' for computing

#summarizing the layers of all models; helps validate the model before training, avoiding errors later.

# from torchsummary import summary
# print(1, summary(model, (3, 224, 224)))
# print(2, summary(model2, (3, 224, 224)))
# print(3, summary(model3, (3, 224, 224))) 

#output shape [-1,c,h,w]
#c->no of filters/ channels
#h,w-> feature map size after operation

In [ ]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
# calculates the loss during training, which will be later used by backpropagation to imporove the models accuracy
optimizer_vgg = torch.optim.Adam(model.parameters())  
optimizer_mobilenet= torch.optim.Adam(model2.parameters()) 
optimizer_resnet = torch.optim.Adam(model3.parameters())  
#adam optimiser is used to optimise the models parameters(weights of the model) to minimise the loss and hence increase the accuracy

### Gradient Descent

In [16]:
data_transforms = {
    'train_loader': transforms.Compose([
        transforms.RandomAffine(degrees=10, translate=(0.05, 0.05), shear=5) , # image ko randomly rotate karne ke liye
        transforms.ColorJitter(hue=0.05, saturation=0.05), # naam jo suggest karta hai
        transforms.RandomHorizontalFlip(), #randomly image ko flip kar deta hai 50% probability saath
        transforms.Grayscale(num_output_channels=1), #image is converted to black and white when =1; when=3 it is converted to fake RGB
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=7)], p=0.2), # randomly applies Gaussian blur with probability=20%;
                                                                                 # kernel size=7->moderate blur; 3->light blur; 15->heavy blur
        transforms.ToTensor(),#image ko convert karta hai pixelated values ye pytorch tensor me ie [0,1]
        transforms.Normalize((0.5,), (0.5,)),#pixelated values from [0,1] to [-1,1]
    ]),
    'validation_loader': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ])
}

In [ ]:
batch_size = 64 
#balances training speed and memory usage, as smaller trains slow and require less gpu, while larger batches train faster but require more GPU memory.
#vgg
train_loader_vgg = torch.utils.data.DataLoader(dataset_vgg, batch_size=batch_size, sampler=train_sampler)
test_loader_vgg = torch.utils.data.DataLoader(dataset_vgg, batch_size=batch_size, sampler=test_sampler)
validation_loader_vgg = torch.utils.data.DataLoader(dataset_vgg, batch_size=batch_size, sampler=validation_sampler)

#mobilenet
train_loader_mobilenet = torch.utils.data.DataLoader(dataset_mobilenet, batch_size=batch_size, sampler=train_sampler)
test_loader_mobilenet = torch.utils.data.DataLoader(dataset_mobilenet, batch_size=batch_size, sampler=test_sampler)
validation_loader_mobilenet = torch.utils.data.DataLoader(dataset_mobilenet, batch_size=batch_size, sampler=validation_sampler)

#resnet
train_loader_resnet = torch.utils.data.DataLoader(dataset_resnet, batch_size=batch_size, sampler=train_sampler)
test_loader_resnet = torch.utils.data.DataLoader(dataset_resnet, batch_size=batch_size, sampler=test_sampler)
validation_loader_resnet = torch.utils.data.DataLoader(dataset_resnet, batch_size=batch_size, sampler=validation_sampler)

In [18]:
from datetime import datetime

def batch_gd(model1, model2, model3, criterion, train_loader_vgg, train_loader_mobilenet, train_loader_resnet, 
             validation_loader_vgg, validation_loader_mobilenet, validation_loader_resnet, epochs):
    # Initialize loss arrays for each model
    train_losses_vgg = np.zeros(epochs)
    train_losses_mobilenet = np.zeros(epochs)
    train_losses_resnet = np.zeros(epochs)

    for e in range(epochs):
        t0 = datetime.now()
        
        # Train VGG16 model
        train_loss_vgg = []
        for inputs, targets in train_loader_vgg:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer_vgg.zero_grad()
            output = model1(inputs)
            loss = criterion(output, targets)
            train_loss_vgg.append(loss.item())
            loss.backward()
            optimizer_vgg.step()
        
        # Calculate average VGG loss
        train_loss_vgg_avg = np.mean(train_loss_vgg)
        train_losses_vgg[e] = train_loss_vgg_avg
        
        # Validate VGG16 model
        validation_loss_vgg = []
        for inputs, targets in validation_loader_vgg:
            inputs, targets = inputs.to(device), targets.to(device)
            output = model1(inputs)
            loss = criterion(output, targets)
            validation_loss_vgg.append(loss.item())
        validation_loss_vgg_avg = np.mean(validation_loss_vgg)
        
        # Train MobileNetV2 model
        train_loss_mobilenet = []
        for inputs, targets in train_loader_mobilenet:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer_mobilenet.zero_grad()
            output = model2(inputs)
            loss = criterion(output, targets)
            train_loss_mobilenet.append(loss.item())
            loss.backward()
            optimizer_mobilenet.step()
        
        # Calculate average MobileNet loss
        train_loss_mobilenet_avg = np.mean(train_loss_mobilenet)
        train_losses_mobilenet[e] = train_loss_mobilenet_avg
        
        # Validate MobileNetV2 model
        validation_loss_mobilenet = []
        for inputs, targets in validation_loader_mobilenet:
            inputs, targets = inputs.to(device), targets.to(device)
            output = model2(inputs)
            loss = criterion(output, targets)
            validation_loss_mobilenet.append(loss.item())
        validation_loss_mobilenet_avg = np.mean(validation_loss_mobilenet)
        
        # Train ResNet18 model
        train_loss_resnet = []
        for inputs, targets in train_loader_resnet:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer_resnet.zero_grad()
            output = model3(inputs)
            loss = criterion(output, targets)
            train_loss_resnet.append(loss.item())
            loss.backward()
            optimizer_resnet.step()
        
        # Calculate average ResNet loss
        train_loss_resnet_avg = np.mean(train_loss_resnet)
        train_losses_resnet[e] = train_loss_resnet_avg
        
        # Validate ResNet18 model
        validation_loss_resnet = []
        for inputs, targets in validation_loader_resnet:
            inputs, targets = inputs.to(device), targets.to(device)
            output = model3(inputs)
            loss = criterion(output, targets)
            validation_loss_resnet.append(loss.item())
        validation_loss_resnet_avg = np.mean(validation_loss_resnet)
        
        dt = datetime.now() - t0
        
        # Print results for each model
        print(f"Epoch : {e+1}/{epochs}")
        print(f"VGG16 - Train Loss: {train_loss_vgg_avg:.3f}, Validation Loss: {validation_loss_vgg_avg:.3f}")
        print(f"MobileNetV2 - Train Loss: {train_loss_mobilenet_avg:.3f}, Validation Loss: {validation_loss_mobilenet_avg:.3f}")
        print(f"ResNet18 - Train Loss: {train_loss_resnet_avg:.3f}, Validation Loss: {validation_loss_resnet_avg:.3f}")
        print(f"Duration: {dt}")
        print("-" * 50)

    return train_losses_vgg, train_losses_mobilenet, train_losses_resnet

### Asli kaam ab hoga 

In [ ]:
train_losses_vgg, train_losses_mobilenet, train_losses_resnet = batch_gd(
    model, model2, model3, 
    criterion, 
    train_loader_vgg, train_loader_mobilenet, train_loader_resnet,
    validation_loader_vgg, validation_loader_mobilenet, validation_loader_resnet, 
    10
)

### Calculating accuracy

In [ ]:
import matplotlib.pyplot as plt

def accuracy(model, loader):
    n_correct = 0
    n_total = 0
    model.cuda()
    for inputs, targets in loader:
        inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)
        n_correct += (predictions == targets).sum().item()
        n_total += targets.shape[0]

    acc = n_correct / n_total
    return acc

# Calculate accuracy for VGG16
train_acc_vgg = accuracy(model, train_loader_vgg)
test_acc_vgg = accuracy(model, test_loader_vgg)
validation_acc_vgg = accuracy(model, validation_loader_vgg)

# Calculate accuracy for MobileNetV2
train_acc_mobilenet = accuracy(model2, train_loader_mobilenet)
test_acc_mobilenet = accuracy(model2, test_loader_mobilenet)
validation_acc_mobilenet = accuracy(model2, validation_loader_mobilenet)

# Calculate accuracy for ResNet18
train_acc_resnet = accuracy(model3, train_loader_resnet)
test_acc_resnet = accuracy(model3, test_loader_resnet)
validation_acc_resnet = accuracy(model3, validation_loader_resnet)

# Print accuracy for each model
print("VGG16 Accuracy:")
print(f"Train Accuracy: {train_acc_vgg:.4f}\nTest Accuracy: {test_acc_vgg:.4f}\nValidation Accuracy: {validation_acc_vgg:.4f}")
print("\nMobileNetV2 Accuracy:")
print(f"Train Accuracy: {train_acc_mobilenet:.4f}\nTest Accuracy: {test_acc_mobilenet:.4f}\nValidation Accuracy: {validation_acc_mobilenet:.4f}")
print("\nResNet18 Accuracy:")
print(f"Train Accuracy: {train_acc_resnet:.4f}\nTest Accuracy: {test_acc_resnet:.4f}\nValidation Accuracy: {validation_acc_resnet:.4f}")

# Plot losses for all models
plt.figure(figsize=(12, 6))
plt.plot(train_losses_vgg, label='VGG16 Train Loss')
plt.plot(train_losses_mobilenet, label='MobileNetV2 Train Loss')
plt.plot(train_losses_resnet, label='ResNet18 Train Loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss Comparison')
plt.show()